Neutral Network

In [1]:
import pandas as pd

dataset= pd.read_csv("Real estate valuation data set.csv")


X= dataset[dataset.columns[2:-1]].values
y= dataset[dataset.columns[-1]].values



# print(X)



Categorical data

In [2]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
X[:,2]=le.fit_transform(X[:,2])



In [3]:
print(X)

[[ 32.       84.87882  10.       24.98298 121.54024]
 [ 19.5     306.5947    9.       24.98034 121.53951]
 [ 13.3     561.9845    5.       24.98746 121.54391]
 ...
 [ 18.8     390.9696    7.       24.97923 121.53986]
 [  8.1     104.8101    5.       24.96674 121.54067]
 [  6.5      90.45606   9.       24.97433 121.5431 ]]


Split

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

Feature Scaling

In [5]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

Neural Network

In [6]:
#init ANN
import tensorflow as tf
ann=tf.keras.models.Sequential()


In [7]:
#adding hidden layer
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [8]:
# second hidden layer
ann.add(tf.keras.layers.Dense(units=7,activation='relu'))

In [9]:
#output layer
ann.add(tf.keras.layers.Dense(units=1,activation='softmax'))

In [10]:
#compiling ANN
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [11]:
#training ANN

ann.fit(X_train,y_train,epochs=100,batch_size=32)


Epoch 1/100
10/10 [==============================] - 0s 778us/step - loss: -2.4666 - accuracy: 0.0000e+00
Epoch 2/100
10/10 [==============================] - 0s 778us/step - loss: -4.9277 - accuracy: 0.0000e+00
Epoch 3/100
10/10 [==============================] - 0s 667us/step - loss: -7.4273 - accuracy: 0.0000e+00
Epoch 4/100
10/10 [==============================] - 0s 667us/step - loss: -10.0425 - accuracy: 0.0000e+00
Epoch 5/100
10/10 [==============================] - 0s 667us/step - loss: -12.7549 - accuracy: 0.0000e+00
Epoch 6/100
10/10 [==============================] - 0s 613us/step - loss: -15.6160 - accuracy: 0.0000e+00
Epoch 7/100
10/10 [==============================] - 0s 666us/step - loss: -18.7379 - accuracy: 0.0000e+00
Epoch 8/100
10/10 [==============================] - 0s 667us/step - loss: -22.1758 - accuracy: 0.0000e+00
Epoch 9/100
10/10 [==============================] - 0s 667us/step - loss: -25.8659 - accuracy: 0.0000e+00
Epoch 10/100
10/10 [====================

In [12]:
#predict
ann.predict(sc.transform([[32,84.87882,5,24.96305,121.51726]]))


1/1 [==============================] - 0s 46ms/step


array([[1.]], dtype=float32)

In [16]:
# predict test 
import numpy as np
y_pred = ann.predict(X_test)
y_pred=(y_pred >0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

4/4 [==============================] - 0s 668us/step
[[ 1.  45.3]
 [ 1.  14.4]
 [ 1.  46. ]
 [ 1.  15.6]
 [ 1.  50.2]
 [ 1.  38.1]
 [ 1.  48.5]
 [ 1.  41.4]
 [ 1.  51.6]
 [ 1.  40.1]
 [ 1.  42. ]
 [ 1.  42.4]
 [ 1.  28.5]
 [ 1.  39.1]
 [ 1.  53.7]
 [ 1.  36.8]
 [ 1.  40.6]
 [ 1.  55.2]
 [ 1.  39.3]
 [ 1.  44.3]
 [ 1.  53.5]
 [ 1.  25.3]
 [ 1.  22.8]
 [ 1.  51.7]
 [ 1.  45.5]
 [ 1.  34.4]
 [ 1.  31.6]
 [ 1.  25.9]
 [ 1.  49.5]
 [ 1.  24.7]
 [ 1.  51.7]
 [ 1.  29.7]
 [ 1.  45.4]
 [ 1.  50.5]
 [ 1.  26.5]
 [ 1.  30.1]
 [ 1.  46.1]
 [ 1.  28.9]
 [ 1.  54.4]
 [ 1.  15. ]
 [ 1.  44.9]
 [ 1.  29.8]
 [ 1.  23.6]
 [ 1.  48. ]
 [ 1.  16.7]
 [ 1.  40.9]
 [ 1.  40.2]
 [ 1.  12.9]
 [ 1.  21.3]
 [ 1.  62.1]
 [ 1.  63.9]
 [ 1.  40.3]
 [ 1.  42.2]
 [ 1.  19.1]
 [ 1.  28.4]
 [ 1.  28.5]
 [ 1.  60.7]
 [ 1.  37.5]
 [ 1.  39.3]
 [ 1.  28.8]
 [ 1.  42.3]
 [ 1.  44.3]
 [ 1.  42.6]
 [ 1.  57.4]
 [ 1.  42.2]
 [ 1.  25.7]
 [ 1.  13.2]
 [ 1.  27. ]
 [ 1.  56.2]
 [ 1.  38.3]
 [ 1.  20.8]
 [ 1.  43.1]
 [ 1.  78.3

In [15]:
#confusion Matrix

from sklearn.metrics import confusion_matrix, accuracy_score
print(y_test,y_pred)
cm= confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[45.3 14.4 46.  15.6 50.2 38.1 48.5 41.4 51.6 40.1 42.  42.4 28.5 39.1
 53.7 36.8 40.6 55.2 39.3 44.3 53.5 25.3 22.8 51.7 45.5 34.4 31.6 25.9
 49.5 24.7 51.7 29.7 45.4 50.5 26.5 30.1 46.1 28.9 54.4 15.  44.9 29.8
 23.6 48.  16.7 40.9 40.2 12.9 21.3 62.1 63.9 40.3 42.2 19.1 28.4 28.5
 60.7 37.5 39.3 28.8 42.3 44.3 42.6 57.4 42.2 25.7 13.2 27.  56.2 38.3
 20.8 43.1 78.3 20.9 32.5 33.4 32.1 19.2 53.  38.2 46.7 24.6 36.2 23.2
 29.3 38.4 37.5 34.3 44.8 54.4 37.9  7.6 29.5 25.3 13.7 30.  30.8 40.6
 20.7 40.8 55.3 40.5 40.8 40.3 33.1 26.2 41.9 18.8 78.  15.6 39.4 41.
 23.5 35.6 47.1 35.6 55.  41.5 37.4 21.8 31.3 36.8 26.6 40.  50. ] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]


ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [18]:
#performance metrice visualization
from sklearn.datasets import load_iris

iris=load_iris()
df=pd.DataFrame(iris.data,columns=iris.feature_names)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3
